# Decision Tree

* 분류(cateorycal)
* 예측(numeric)

## 기본원리

|구분|설명|수식|
|--|--|--|
|지니계수|클래스의 구분을 더욱 확실히 하게끔 움직임(순수도의 관점)|$Gini(D) = 1 - \sum_{i=1}^{n} (p_i^2)$|
|엔트로피|균형잡힌 클래스를 유도|$Entropy(D) = - \sum_{i=1}^{n} (p_i \log_2(p_i))$|

위의 계수간의 관점 차이를 이해하기 위해선 이진분류부터 이해할 필요가 있어보임           
그렇다면 우리는 decision tree의 손실함수를 이해해야한다         
`information gain`의 개념을 살펴보자

`entropy`는 이진분류일 경우에 효율적이며, 지니계수는 다중분류일 경우에도 사용가능함이 장점이다

### 과정(참고)
- step1. 각각의 노드에서의 계수의 값을 최소로 만들기 위해 계산을 실시

우리는 위의 계수 차이를 이해하기 위해 챃 이진분류 모델을 보며 이해해보자

### 분류모델

* 사용가능 알고리즘

|알고리즘|평가지수|비고|
|--|--|--|
|ID3|엔트로피|사용하지 않음|
|c4.5|엔트로피|ID3의 확장판|
|CHAID|카이제곱 통계량|
|CART|지니계수|현재 파이썬의 sklearn.tree 알고리즘|

> C4.5모델은 `지니계수`도 사용가능하며, Prunig 등 과적합 방지기능까지 탑제되어있다          
> 또한 C5모델로 한층 업그레이드 되어있으므로 사용해보자

### 회귀모델

|알고리즘|평가지수|
|--|--|
|CHAID|F-통계량|
|CART|분산감소량|

> 위에서 `CART`만 유일한 이진분류 모델 기반이다

### 질문선택방식

|구분|특징 및 설명|
|--|--|
|정보이득|(entropy) 질문전 불확실성-질문후 불확실성|
||(Geni) 질문전 지니불순도-질문후 지니불순도|
|카이제곱검정|분할 후 클래스 분포와 기대 클래스 분포간의 차이|

![Alt text](decision_tree_image/image.png)

이진분류를 가정했을 때의 그래프

## 분류모델의 실습

In [38]:
import sklearn.tree as sktree
import pandas as pd
import scipy as sp

In [39]:
Data=pd.read_csv('C:/Users/User/Desktop/glass.csv')
data=pd.DataFrame(Data)

In [40]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214 entries, 0 to 213
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   RI      214 non-null    float64
 1   Na      214 non-null    float64
 2   Mg      214 non-null    float64
 3   Al      214 non-null    float64
 4   Si      214 non-null    float64
 5   K       214 non-null    float64
 6   Ca      214 non-null    float64
 7   Ba      214 non-null    float64
 8   Fe      214 non-null    float64
 9   Type    214 non-null    int64  
dtypes: float64(9), int64(1)
memory usage: 16.8 KB


#### target & features

In [41]:
target=["Type"]
feature=list(data.columns.difference(target))

In [42]:
data[target].nunique()

Type    6
dtype: int64

이때 이미지 등 뒤에 문제가 생기므로 category로 바꿔줘야함

In [43]:
data['Type']=data["Type"].apply(str)

### 참고!
파이썬은 `CART`알고리즘만 제공한다고 알려져있다. 따라서, KNIME을 통해 나머지 decision Tree를 구현하자.

In [44]:
import numpy as np
np.random.seed(10)
import sklearn.model_selection as skmod
train,test=skmod.train_test_split(data,train_size=0.8,random_state=10)

### 모델 구축
> 이때 종속변수의 종류가 6이므로 지니계수를 이용

In [45]:
tree_model=sktree.DecisionTreeClassifier(criterion='gini',max_depth=5,random_state=10)

In [46]:
tree_model.fit(X=train[feature],y=train[target])

DecisionTreeClassifier(max_depth=5, random_state=10)

In [47]:
pred_result=tree_model.predict(X=test[feature])

In [48]:
import sklearn.metrics as skmet

In [49]:
pred_result

array(['1', '1', '5', '3', '1', '6', '2', '5', '1', '3', '1', '5', '1',
       '7', '1', '2', '6', '3', '3', '2', '3', '2', '6', '3', '2', '5',
       '2', '2', '1', '5', '1', '2', '2', '6', '2', '7', '2', '7', '2',
       '1', '1', '2', '1'], dtype=object)

In [50]:
skmet.accuracy_score(test[target],pred_result)

0.627906976744186

### 결과해석
> 해당 model의 criterion을 'entropy'로 설정할 경우, accuaracy가 55%로 크게 떨어짐

In [51]:
pd.DataFrame(list(zip(tree_model.feature_names_in_,tree_model.feature_importances_)))

,0,1
0,Al,0.152640
1,Ba,0.248269
2,Ca,0.000000
3,Fe,0.000000
4,K,0.000000
5,Mg,0.318229
6,Na,0.110784
7,RI,0.112352
8,Si,0.057724


Ca,Fe,K 의 값이 크게 영향을 주지 못함
> 이 값들을 제거하고 만들었어도 괜찮았을 듯         
> feature_inportances : 계수(지니,엔트로피)에 영향을 준 정도 * 샘플 size

#### 모델 시각화

In [52]:
tree_image=sktree.export_graphviz(tree_model,
                       out_file=None,
                       feature_names=tree_model.feature_names_in_,
                       class_names=tree_model.classes_,
                       filled=True, rounded=True,
                       special_characters=True)

In [53]:
import graphviz
import IPython.display as display

In [54]:
graph=graphviz.Source(tree_image)
graph.format='png'
image_path='./decision_tree.png'
graph.render(filename=image_path)

'decision_tree.png.png'

![Alt text](decision_tree_image/decision_tree.png.png)

#### Pruning 등 과적합 방지 방법

사전 가지치기 VS 사후 가지치기($\alpha$값의 변동)

* 사전 가치치기

|파라미터|설명|
|--|--|
|min_samples_split|노드를 분할하기 위한 최소 샘플 수|
|min_samples_leaf|리프노드의 최소 샘플 수|

가지치기를 실시
> 임의로 가지치기 한것이며(지나치게 적은 sample size의 노드를 줄이기 위해), 실제론 이미지를 하나씩 보면서 분석하고 결정해야함

In [56]:
pre_tree=sktree.DecisionTreeClassifier(criterion='gini',
                              max_depth=5,
                              min_samples_split=10,
                              min_samples_leaf=5)

In [57]:
pre_tree.fit(X=train[feature],y=train[target])

DecisionTreeClassifier(max_depth=5, min_samples_leaf=5, min_samples_split=10)

모델 평가 및 시각화

In [59]:
pre_result=pre_tree.predict(test[feature])
skmet.accuracy_score(test[target],pre_result)

0.6046511627906976

In [61]:
pre_tree_image=sktree.export_graphviz(pre_tree,out_file=None,
                                      feature_names=pre_tree.feature_names_in_,
                                      class_names=pre_tree.classes_,
                                      filled=True,rounded=True,
                                      special_characters=True)

In [63]:
pre_image=graphviz.Source(pre_tree_image)
pre_image.format='png'
pre_path='./pre_pruning.png'
pre_image.render(pre_path)

'pre_pruning.png.png'

![Alt text](decision_tree_image/pre_pruning.png.png)